# Agent Hand-Off

Hand-off is when control transfers entirely from one agent to another. Unlike delegation where the parent retains control, hand-off means the original agent's job is done once it decides who should handle the request.

In [ ]:
from enum import Enum
from pydantic import BaseModel, Field
from agentic_patterns.core.agents import get_agent, run_agent

## The Scenario

A customer support system routes requests to specialists. The triage agent classifies the request and decides which specialist handles it. Once handed off, the triage agent is no longer involved.

## Classification Output

The triage agent produces a structured classification. The enum ensures routing decisions map to known specialists.

In [ ]:
class RequestCategory(str, Enum):
    BILLING = "billing"
    TECHNICAL = "technical"


class TriageResult(BaseModel):
    category: RequestCategory = Field(description="The category of the request")
    summary: str = Field(description="Brief summary of the customer's issue")

## The Triage Agent

The triage agent's only job is classification. It does not solve the problem, it routes it.

In [ ]:
triage_agent = get_agent(
    output_type=TriageResult,
    system_prompt="""You are a customer support triage agent.
Classify incoming requests as either billing or technical.
Billing: payments, invoices, subscriptions, refunds, pricing.
Technical: bugs, errors, how-to questions, feature requests, integrations."""
)

## The Specialist Agents

Each specialist handles a specific category. They receive the customer's request and provide a complete response.

In [ ]:
billing_agent = get_agent(
    system_prompt="""You are a billing support specialist.
Help customers with payments, invoices, subscriptions, and refunds.
Be helpful and provide clear next steps."""
)

technical_agent = get_agent(
    system_prompt="""You are a technical support specialist.
Help customers with bugs, errors, how-to questions, and integrations.
Provide clear explanations and actionable solutions."""
)

## The Hand-Off Logic

Routing is explicit in application code. The triage agent classifies, then the orchestrator hands off to the appropriate specialist. This is not a tool call - it's a control flow decision.

In [ ]:
async def handle_support_request(customer_message: str) -> str:
    """Route a customer request to the appropriate specialist."""
    
    # Step 1: Triage classifies the request
    triage_run, _ = await run_agent(triage_agent, customer_message)
    classification = triage_run.result.output
    
    print(f"Triage: {classification.category.value} - {classification.summary}")
    
    # Step 2: Hand off to specialist (triage is done)
    match classification.category:
        case RequestCategory.BILLING:
            specialist = billing_agent
        case RequestCategory.TECHNICAL:
            specialist = technical_agent
    
    # The specialist handles the request completely
    specialist_run, _ = await run_agent(specialist, customer_message)
    return specialist_run.result.output

## Example: Billing Request

In [ ]:
billing_request = "I was charged twice for my subscription last month. Can I get a refund?"

response = await handle_support_request(billing_request)
print(f"\nResponse:\n{response}")

## Example: Technical Request

In [ ]:
technical_request = "The API returns a 500 error when I try to upload files larger than 10MB."

response = await handle_support_request(technical_request)
print(f"\nResponse:\n{response}")